In [ ]:
# md5 : 속도보다 보안 but 보안도 이제는 떨어짐
import hashlib
def md5(file1):
    md5h = hashlib.md5()
    with open(file1, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            md5h.update(chunk)
    return md5h.hexdigest()

In [1]:
import os
from tqdm.notebook import tqdm
import pandas as pd
import file_function as ff
import re
import binascii

In [2]:
def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

In [3]:
def file_info(path):

    dict_sc = {}  # 중복파일명 숫자를 카운트 할 딕셔너리
    dict_sm = {}
    dict_scm = {}

    cols = ['size', 'crc32', 'mtime',  'sell', 'root', 'stem', 'ext', 'length']
    df = pd.DataFrame(columns=cols)
    
    df_matching = pd.read_excel(
        r'D:\workspace\python_work\파일\매각사 이름매칭.xlsx')

    for root, __dirs__, files in tqdm(os.walk(path)):
        
        for f in files:
            fullname = os.path.join(root, f)
            #key
            size = str(os.path.getsize(fullname))
            #value
            mtime = str(os.path.getmtime(fullname))
            crc32 = str(crc32_checksum(fullname))
            sell = ""
            for __index__, row in df_matching.iterrows():
                if re.search(str(row[0]), str(root)):  
                    sell = row[1]
                    break            
            stem = os.path.splitext(f)[0]
            ext = os.path.splitext(f)[1]
            length = len(stem)
            
            temp1 = {"size":str(size)}
            temp2 = {"crc32": str(crc32), "mtime":str(mtime), "sell":sell, "root":root, "stem":stem, "ext":ext, "length":length}
            
            # 고유한 size, crc를 키로 하는 딕셔너리 만들기 -> mtime 다른 경우 있는지 체크
            sc = size + crc32
            if sc not in dict_sc:
                dict_sc[sc] = [temp2]
            else:
                dict_sc[sc].append(temp2)
            
            sm = size + mtime
            if sm not in dict_sm:
                dict_sc[sm] = [temp2]
            else:
                dict_sc[sm].append(temp2)

            scm = size + crc32 + mtime
            if scm not in dict_scm:
                dict_sc[scm] = [temp2]
            else:
                dict_sc[scm].append(temp2)

            # size도 하나의 col로 df 만들기
            temp1.update(temp2)
            temp_df = pd.DataFrame([temp1])
            
            df = pd.concat([df, temp_df], ignore_index=True)
    
    df.to_excel(r"C:\Users\SL\Desktop\파일정보.xlsx")
            
    return dict_sc, dict_sm, dict_scm

In [4]:
path = r'\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채권양도통지서'
dict_sc, dict_sm, dict_scm = file_info(path)

0it [00:00, ?it/s]

In [11]:
len(dict_sc.keys())

54134

In [8]:
temp_df

NameError: name 'temp_df' is not defined

In [12]:
df = pd.read_excel(r'C:\Users\SL\Desktop\파일정보.xlsx')
len(df)

,Unnamed: 0,size,crc32,mtime,sell,root,stem,ext,length
0,0,512412,D080D821,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)로뎀 615-81 디엔피양통,.pdf,18
1,1,524647,0081BF65,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)로뎀의 보증인 장금현 520920 디엔피양통,.pdf,27
2,2,495997,3A7BCF24,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보 605-81 양통,.pdf,18
3,3,496701,13A0FB4F,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보의 보증인 윤명식 631108 디엔피양통,.pdf,30
4,4,498615,21C81D99,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보의 보증인 한상봉 670617 디엔피양통,.pdf,30


In [20]:
len(df)

23321

In [24]:
s = df.duplicated(['size'], keep=False)
s.name = "s"
s.value_counts()
# 사이즈가 같은 건 엄청나게 많다. 충분조건에 불과하다

True     20577
False     2744
Name: s, dtype: int64

In [21]:
sc = df.duplicated(['size', 'crc32'], keep=False)
sc.name = "sc"
sc.value_counts()
# 더블킬러보다 한세트 더 많은데, 그게 맞았다.
# 수정일이 다르지만 같은 파일인 경우는 약 0.22% 
# sc로 가면 되겠다.

False    12812
True     10509
Name: sc, dtype: int64

In [22]:
sm = df.duplicated(['size', 'mtime'], keep=False)
sm.name = "sm"
sm.value_counts()

False    12834
True     10487
Name: sm, dtype: int64

In [23]:
scm = df.duplicated(['size', 'crc32', 'mtime'], keep=False)
scm.name = "scm"
scm.value_counts()

False    12834
True     10487
Name: scm, dtype: int64

In [25]:
df2 = pd.concat([df,sc, sm, scm], axis=1)
df2.head()

,Unnamed: 0,size,crc32,mtime,sell,root,stem,ext,length,sc,sm,scm
0,0,512412,D080D821,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)로뎀 615-81 디엔피양통,.pdf,18,True,True,True
1,1,524647,0081BF65,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)로뎀의 보증인 장금현 520920 디엔피양통,.pdf,27,True,True,True
2,2,495997,3A7BCF24,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보 605-81 양통,.pdf,18,True,True,True
3,3,496701,13A0FB4F,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보의 보증인 윤명식 631108 디엔피양통,.pdf,30,True,True,True
4,4,498615,21C81D99,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보의 보증인 한상봉 670617 디엔피양통,.pdf,30,True,True,True


In [26]:
df2.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.head()

,size,crc32,mtime,sell,root,stem,ext,length,sc,sm,scm
0,512412,D080D821,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)로뎀 615-81 디엔피양통,.pdf,18,True,True,True
1,524647,0081BF65,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)로뎀의 보증인 장금현 520920 디엔피양통,.pdf,27,True,True,True
2,495997,3A7BCF24,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보 605-81 양통,.pdf,18,True,True,True
3,496701,13A0FB4F,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보의 보증인 윤명식 631108 디엔피양통,.pdf,30,True,True,True
4,498615,21C81D99,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보의 보증인 한상봉 670617 디엔피양통,.pdf,30,True,True,True


In [27]:
#df_sc = df2[df2["sc"]]
#df_sc.to_excel(r"C:\Users\SL\Desktop\파일정보_sc.xlsx")

,size,crc32,mtime,sell,root,stem,ext,length,sc,sm,scm
0,512412,D080D821,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)로뎀 615-81 디엔피양통,.pdf,18,True,True,True
1,524647,0081BF65,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)로뎀의 보증인 장금현 520920 디엔피양통,.pdf,27,True,True,True
2,495997,3A7BCF24,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보 605-81 양통,.pdf,18,True,True,True
3,496701,13A0FB4F,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보의 보증인 윤명식 631108 디엔피양통,.pdf,30,True,True,True
4,498615,21C81D99,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)마라솔정보의 보증인 한상봉 670617 디엔피양통,.pdf,30,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
23316,497065,D432F6A2,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,하바울(하경숙) 691013 디엔피양통,.pdf,21,True,True,True
23317,502165,A178C21B,1.624238e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,한규수 370203 디엔피양통,.pdf,16,True,True,True
23318,494511,52E6834E,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,한덕주 640320 디엔피양통,.pdf,16,True,True,True
23319,512672,A85F0234,1.624235e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,황용숙 570915 디엔피양통,.pdf,16,True,True,True


In [31]:
df_sc=pd.read_excel(r"C:\Users\SL\Desktop\파일정보_sc.xlsx")
df_sc.head(2)

,Unnamed: 0,size,crc32,mtime,sell,root,stem,ext,length,sc,full
0,0,512412,D080D821,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)로뎀 615-81 디엔피양통,.pdf,18,True,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...
1,1,524647,0081BF65,1.624237e+09,디엔피에이엠씨대부㈜,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,(주)로뎀의 보증인 장금현 520920 디엔피양통,.pdf,27,True,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...


In [33]:
df_dk = pd.read_excel(r"C:\Users\SL\Desktop\파일정보_더블킬러.xlsx")
df_dk.head(1)

,index,selection,filename,root,size,msize,crc32,full
0,431,0,20411246 라기문_양통 (1).pdf,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...,39312,2021-01-25 오후 3:38:58,22B36CA0,\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채...


In [36]:
set_sc = set(df_sc.full)
set_dk = set(df_dk.full)
print(len(set_sc))
print(len(set_dk))


10509
10507


In [37]:
set_sc - set_dk

{'\\\\192.168.0.75\\sollim광주\\광주 공유폴더\\(1.DNP_헬프)\\3.채권양도통지서\\20416181 김영보_양통.pdf',
 '\\\\192.168.0.75\\sollim광주\\광주 공유폴더\\(1.DNP_헬프)\\3.채권양도통지서\\디앤피양도통지서\\김영보.pdf'}

In [ ]:
only_dk = []
only_py = []
[only_dk.append(x) for x in list_dk if x not in list_py]
[only_py.append(x) for x in list_py if x not in list_dk]

In [38]:
set_dk - set_sc

set()